# python 中的 GIL

## GIL(global interpreter lock)全局解释器锁（cpython）

为了使得线程运行安全，尤其是多线程，python 会在解释器上加一把锁， 使得使得同一个时刻只有一个线程在一个cpu上执行字节码，无法将多个线程映射到多个CPU上执行，无法体现多CPU优势。

- python中一个线程对应c语言的一个线程

- python运行时的过程是把py文件编译成字节码

dis库是python(默认的CPython)自带的一个库,可以用来分析字节码

Python代码是先被编译为Python字节码后，再由Python虚拟机来执行Python字节码（pyc文件主要就是用于存储字节码指令 的）。一般来说一个Python语句会对应若干字节码指令，Python的字节码是一种类似汇编指令的中间语言，但是一个字节码指令并不是对应一个机器指 令（二进制指令），而是对应一段C代码，而不同的指令的性能不同，所以不能单独通过指令数量来判断代码的性能，而是要通过查看调用比较频繁的指令的代码来 确认一段程序的性能。

In [19]:
import dis

def add(a):
    a = a + 1
    return a

print(dis.dis(add))

  4           0 LOAD_FAST                0 (a)
              2 LOAD_CONST               1 (1)
              4 BINARY_ADD
              6 STORE_FAST               0 (a)

  5           8 LOAD_FAST                0 (a)
             10 RETURN_VALUE
None


有了GIL，是不是意味编写多线程的时候是安全的，不用去考虑线程间的同步呢？实际上不是的，会在某些时刻释放

In [22]:
import threading

total = 0

def add():
    global total
    for i in range(1000000):
        total += 1
    
def sub():
    global total
    for i in range(1000000):
        total -= 1
    
thread1 = threading.Thread(target=add)
thread2 = threading.Thread(target=sub)
thread1.start()
thread2.start()
thread1.join()
thread1.join()
print(total)

300396


每次运行的结果都不一样，这表明 GIL 会释放的。

GIL 这把锁分配给某一个线程之后，并不是说这个线程执行完了之后才会释放，再交给另一个线程。
它不是整个过程的完全占有，它会在适当的时候释放，另外一个线程就可以得到运行。GIL 结合了字节码的行数/时间片划分，比如执行了100行或者1000行之后，它会释放。

- GIL 会根据执行的字节码行数以及时间片释放
- GIL在遇到IO操作时候主动释放。

正是这个特性，使得python 多线程在IO操作频繁的情况下，非常适用的。

## 多线程编程

对于io操作来说，多线程和多进程性能差别不大

多任务可以由多进程完成，也可以由一个进程内的多线程完成。进程是由若干线程组成的，一个进程至少有一个线程。由于线程是操作系统直接支持的执行单元，因此，高级语言通常都内置多线程的支持，Python也不例外，并且，Python的线程是真正的Posix Thread，而不是模拟出来的线程。Python的标准库提供了两个模块：_thread和threading，_thread是低级模块，threading是高级模块，对_thread进行了封装。绝大多数情况下，我们只需要使用threading这个高级模块。

启动一个线程就是把一个函数传入并创建Thread实例，然后调用start()开始执行：

In [26]:
import time,threading

def loop():
    print('thread is running...'.format(threading.current_thread().name))
    n = 0
    while n < 5:
        n = n + 1
        print('thread {} >>> {}'.format(threading.current_thread().name,n))
        time.sleep(1)
    print('thred {} ended.'.format(threading.current_thread().name))
    
print('thread {} is running...'.format(threading.current_thread().name))

t = threading.Thread(target=loop,name='LoopThread')
#t = threading.Thread(target=loop)
t.start()
t.join()
print('thread {} ended.'.format(threading.current_thread().name))

thread MainThread is running...
thread is running...
thread LoopThread >>> 1
thread LoopThread >>> 2
thread LoopThread >>> 3
thread LoopThread >>> 4
thread LoopThread >>> 5
thred LoopThread ended.
thread MainThread ended.


- 由于任何进程默认就会启动一个线程，我们把该线程称为主线程，主线程又可以启动新的线程。

- Python的threading模块有个current_thread()函数，它永远返回当前线程的实例。

- 主线程实例的名字叫 MainThread，子线程的名字在创建时指定, `t = threading.Thread(target=loop,name='LoopThread'`我们用 LoopThread 命名子线程。如果不起，python 会自动给线程命名为Thread-1，Thread-2……

### 多线程编写方式一：通过 Thread 类实例化

- target 目标函数名
- args 函数参数

In [16]:
import time,threading

def get_detail_html(url):
    print('get detail html started')
    time.sleep(2)
    print('get detail html end')
    
def get_detail_url(url):
    print('get detail url started\n')
    time.sleep(2)
    print('get detail url end')
    
if __name__ == '__main__':
    thread1 = threading.Thread(target=get_detail_html,args=('',)) # 用 thread 类实例化
    thread2 = threading.Thread(target=get_detail_url,args=('',)) 
    start_time = time.time()
    thread1.start()
    thread2.start()
    print('finished in {} s'.format(time.time()-start_time))

get detail html started
get detail url started
finished in 0.019019126892089844 s

get detail html end
get detail url end


问题：时间消耗0.02s？

正常理解，两个线程并行，时间消耗应该是2秒,为什么是0.01 秒

解答：我们创建了2个线程，thread1和thread2， 但其实这里有3个线程，那就是主线程，除掉线程的代码，剩下的
```
print('finished in {} s'.format(time.time()-start_time))
```
是在主线程里面运行的。 【可在 print 处 debug,Debugger 里面的 Frames,可以看到它有3个线程】

所以这里是3个线程并行，thread1开始，thread2开始，主线程开始(即`print('finished in {} s'.format(time.time()-start_time))`)

这里可以看出，主线程执行完，并没有退出，而是等待 thread1和thread2执行完才退出


#### 需求一：当主线程退出的时候，子线程kill掉。
（jupyter notebook 结果有误）

##### thread.setDaemon(True)

In [ ]:
import time,threading

def get_detail_html(url):
    print('get detail html started')
    time.sleep(2)
    print('get detail html end')
    
def get_detail_url(url):
    print('get detail url started')
    time.sleep(2)
    print('get detail url end')
    
if __name__ == '__main__':
    thread1 = threading.Thread(target=get_detail_html,args=('',)) # 用 thread 类实例化
    thread2 = threading.Thread(target=get_detail_url,args=('',)) 
    thread1.setDaemon(True) # 守护线程
    thread2.setDaemon(True) # 守护线程
    start_time = time.time()
    thread1.start()
    thread2.start()
    print('finished in {} s'.format(time.time()-start_time))

结果如下
```
get detail html started
get detail url started
finished in 0.0 s
```

在启动线程前设置 thread.setDaemon(True)，就是设置该线程为守护线程，表示该线程是不重要的,进程退出时不需要等待这个线程执行完成。

这样做的意义在于：避免子线程无限死循环，导致退不出程序。

上述例子，主线程执行完print后就退出了，不用等待 thread1 和 thread2完成

**如果设置1个线程为守护线程，会如何？**

In [ ]:
import time,threading

def get_detail_html(url):
    print('thread1: get detail html started')
    time.sleep(2)
    print('thread1: get detail html end')
    
def get_detail_url(url):
    print('thread2: get detail url started')
    time.sleep(2)
    print('thread2: get detail url end')
    
if __name__ == '__main__':
    thread1 = threading.Thread(target=get_detail_html,args=('',)) # 用 thread 类实例化
    thread2 = threading.Thread(target=get_detail_url,args=('',)) 
    #thread1.setDaemon(True) # 守护线程
    thread2.setDaemon(True) # 守护线程
    start_time = time.time()
    thread1.start()
    thread2.start()
    print('finished in {} s'.format(time.time()-start_time))

结果如下
```
thread1: get detail html started
thread2: get detail url started
finished in 0.001001596450805664 s
thread2: get detail url end
thread1: get detail html end
```

全部打印，因为 thread1 不是守护线程，主线程执行完 print 需要等待 thread1 执行完。
而 thread2 是守护线程，只有当主线程退出才会被 kill 掉。由于thread1和thread2 需要运行完成的时间是一样的，所以全部完成。


**为了说明区别，把thread2 改成 time.sleep(4)**

In [ ]:
import time,threading

def get_detail_html(url):
    print('get detail html started')
    time.sleep(2)
    print('get detail html end')
    
def get_detail_url(url):
    print('get detail url started')
    time.sleep(4)
    print('get detail url end')
    
if __name__ == '__main__':
    thread1 = threading.Thread(target=get_detail_html,args=('',)) # 用 thread 类实例化
    thread2 = threading.Thread(target=get_detail_url,args=('',)) 
    #thread1.setDaemon(True) # 守护线程
    thread2.setDaemon(True) # 守护线程
    start_time = time.time()
    thread1.start()
    thread2.start()
    print('finished in {} s'.format(time.time()-start_time))

结果如下
```
thread1: get detail html started
thread2: get detail url started
finished in 0.0009961128234863281 s
thread1: get detail html end
```

因为 thread1 不是守护线程，主线程执行完 print 需要等待 thread1 执行完。
而 thread2 是守护线程，只有当主线程退出才会被 kill 掉。由于thread1 运行完需要2秒，而 thread2 运行完需要4秒。

主线程执行完 print，等待 thread1 执行完(2秒) 就可以退出，而此时 thread2 并没执行完成。


#### 需求二：等待线程完成，之后再往下执行
（jupyter notebook 结果有误）

##### thread.join()

主线程A中，创建了子线程B，并且在主线程A中调用了B.join()，那么，主线程A会在调用的地方等待，直到子线程B完成操作后，才可以接着往下执行，那么在调用这个线程时可以使用被调用线程的join方法。

1. 阻塞主进程，专注于执行多线程中的程序。

2. 多线程多join的情况下，依次执行各线程的join方法，前头一个结束了才能执行后面一个。

3. 无参数，则等待到该线程结束，才开始执行下一个线程的join。

4. 参数timeout为线程的阻塞时间，如 timeout=2 就是罩着这个线程2s 以后，就不管他了，继续执行下面的代码。

In [ ]:
import time,threading

def get_detail_html(url):
    print('get detail html started')
    time.sleep(2)
    print('get detail html end')
    
def get_detail_url(url):
    print('get detail url started')
    time.sleep(4)
    print('get detail url end')
    
if __name__ == '__main__':
    thread1 = threading.Thread(target=get_detail_html,args=('',)) # 用 thread 类实例化
    thread2 = threading.Thread(target=get_detail_url,args=('',)) 
    start_time = time.time()
    thread1.start()
    thread2.start()
    thread1.join()
    thread2.join()
    print('finished in {} s'.format(time.time()-start_time))

结果如下
```
get detail html started
get detail url started
get detail html end
get detail url end
finished in 4.00110387802124 s
```
- 运行时间是4s,而不是2个线程这和，说明这里是并发
- 主线程 print 是等待 thread1 和 thread2 完成后才执行

这个多线程写法还是比较复杂，代码量小的时候还可以适用。

当代码量大，内部逻辑比较复杂，这种写法是不适用的。所以 python 有另一种常用的多线程写法。

### 多线程编写方式二：继承 Thread 类

**重载 run 方法，不是 start() 方法**

In [ ]:
import time,threading

class GetDetailHtml(threading.Thread):
    
    
    def run(self):
        print('get detail html started')
        time.sleep(2)
        print('get detail html end')
    
class GetDetailUrl(threading.Thread):
    def run(self):
        print('get detail url started')
        time.sleep(4)
        print('get detail url end')
        
if __name__ == '__main__':
    thread1 = GetDetailHtml()
    thread2 = GetDetailUrl()
    
    start_time = time.time()
    thread1.start()
    thread2.start()
    thread1.join()
    thread2.join()
    print('finished in {} s'.format(time.time()-start_time))

结果和上面一样。

## 线程间通信

### 线程间通信方式一：共享全局变量

- 函数外全局变量
- 函数参数
- 参数放到一个py文件中

In [ ]:
import time,threading

detail_url_list = []

def get_detail_html():
    # 爬取文章详情页
    global detail_url_list
    detail_url_list.pop()
    print('get detail html started')
    time.sleep(2)
    print('get detail html end')

def get_detail_url():
    # 爬取文章列表页
    print('get detail url started')
    time.sleep(4)
    for i in range(20):
        detail_url_list.append('http://threadingtest.com/{}'.format(i))
    print('get detail url end')
    
if __name__ == '__main__':
    thread_detail_url = threading.Thread(target=get_detail_url) 
    for i in range(10):
        html_thread = threading.Thread(target=get_detail_html)
        html_thread.start()
    
#     start_time = time.time()
#     thread1.start()
#     thread2.start()
#     thread1.join()
#     thread2.join()
#     print('finished in {} s'.format(time.time()-start_time))

In [33]:
from queue import Queue
import time,threading

def get_detail_html(queue):
    # 爬取文章详情页
    while True:
        url = queue.get()
        global detail_url_list
        detail_url_list.pop()
        print('get detail html started')
        time.sleep(2)
        print('get detail html end')

def get_detail_url(queue):
    # 爬取文章列表页
    while True:
        print('get detail url started')
        time.sleep(4)
        for i in range(20):
            queue.put('http://threadingtest.com/{}'.format(i))
        print('get detail url end')
    
if __name__ == '__main__':
    detail_url_queue = Queue(maxsize =1000)
    thread_detail_url = threading.Thread(target=get_detail_url,args=(detail_url_queue,)) 
    for i in range(10):
        html_thread = threading.Thread(target=get_detail_html,args=(detail_url_queue,))
        html_thread.start()

## 线程同步 Lock

**多线程和多进程最大的不同在于**

- 多进程中，同一个变量，各自有一份拷贝存在于每个进程中，互不影响。

- 多线程中，所有变量都由所有线程共享，所以，任何一个变量都可以被任何一个线程修改。

因此，线程之间共享数据最大的危险在于多个线程同时改一个变量，把内容给改乱了。

来看看多个线程同时操作一个变量怎么把内容给改乱了：

In [34]:
import time, threading
import dis

# 假定这是你的银行存款
balance = 0

def change_it(n):
    # 先存后取，结果应该为0:
    global balance
    balance = balance + n
    balance = balance - n
    
def run_thread(n):
    for i in range(1000000):
        change_it(n)
        
t1 = threading.Thread(target=run_thread, args=(5,))
t2 = threading.Thread(target=run_thread, args=(8,))
t1.start()
t2.start()
t1.join()
t2.join()
print(balance)

print(dis.dis(change_it))

16
 10           0 LOAD_GLOBAL              0 (balance)
              2 LOAD_FAST                0 (n)
              4 BINARY_ADD
              6 STORE_GLOBAL             0 (balance)

 11           8 LOAD_GLOBAL              0 (balance)
             10 LOAD_FAST                0 (n)
             12 BINARY_SUBTRACT
             14 STORE_GLOBAL             0 (balance)
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE
None


由于以下两个特性

- 为了使得线程运行安全，尤其是多线程，python 会在解释器上加一把锁 GIL， 使得使得同一个时刻只有一个线程在一个cpu上执行字节码

- GIL 这把锁分配给某一个线程之后，并不是说这个线程执行完了之后才会释放，再交给另一个线程。 它不是整个过程的完全占有，它会在适当的时候释放，另外一个线程就可以得到运行。

线程 t1、t2 是交替执行的。


我们定义了一个共享变量balance，初始值为0，并且启动两个线程，先存后取，理论上结果应该为0，但是，由于线程的调度是由操作系统决定的，当t1、t2交替执行时，只要循环次数足够多，balance的结果就不一定是0了。

原因是因为高级语言的一条语句在CPU执行时是若干条语句，即使一个简单的计算：(反编译字节码也可以看出)

`balance = balance + n`

也分两步：

1. 计算balance + n，存入临时变量中；
2. 将临时变量的值赋给balance。

也就是可以看成：

```
x = balance + n
balance = n
```
由于x是局部变量，两个线程各自都有自己的x，当代码正常执行时：

```
t1: x1 = balance + 5 # x1 = 0 + 5 = 5
t1: balance = x1     # balance = 5
t1: x1 = balance - 5 # x1 = 5 - 5 = 0
t1: balance = x1     # balance = 0

t2: x2 = balance + 8 # x2 = 0 + 8 = 8
t2: balance = x2     # balance = 8
t2: x2 = balance - 8 # x2 = 8 - 8 = 0
t2: balance = x2     # balance = 0

结果 balance = 0
```
但是t1和t2是交替运行的，如果操作系统以下面的顺序执行t1、t2：

```
初始值 balance = 0

t1: x1 = balance + 5  # x1 = 0 + 5 = 5

t2: x2 = balance + 8  # x2 = 0 + 8 = 8
t2: balance = x2      # balance = 8

t1: balance = x1      # balance = 5
t1: x1 = balance - 5  # x1 = 5 - 5 = 0
t1: balance = x1      # balance = 0

t2: x2 = balance - 8  # x2 = 0 - 8 = -8
t2: balance = x2   # balance = -8

结果 balance = -8
```

究其原因，是因为修改balance需要多条语句，而执行这几条语句时，线程可能中断，从而导致多个线程把同一个对象的内容改乱了。

两个线程同时一存一取，就可能导致余额不对，你肯定不希望你的银行存款莫名其妙地变成了负数，所以，我们必须确保一个线程在修改balance的时候，别的线程一定不能改。

如果我们要确保balance计算正确，就要给change_it()上一把锁，当某个线程开始执行change_it()时，我们说，该线程因为获得了锁，因此其他线程不能同时执行change_it()，只能等待，直到锁被释放后，获得该锁以后才能改。由于锁只有一个，无论多少线程，同一时刻最多只有一个线程持有该锁，所以，不会造成修改的冲突。创建一个锁就是通过threading.Lock()来实现：

In [30]:
import time, threading

# 假定这是你的银行存款
balance = 0
lock = threading.Lock()

def change_it(n):
    # 先存后取，结果应该为0:
    global balance
    balance = balance + n
    balance = balance - n
    
def run_thread(n):
    for i in range(1000000):
        # 先获取锁
        lock.acquire()
        try:
            change_it(n)
        finally:
            # 改完了一定要释放锁
            lock.release()
            
t1 = threading.Thread(target=run_thread, args=(5,))
t2 = threading.Thread(target=run_thread, args=(8,))
t1.start()
t2.start()
t1.join()
t2.join()
print(balance)

0


当多个线程同时执行lock.acquire()时，只有一个线程能成功地获取锁，然后继续执行代码，其他线程就继续等待直到获得锁为止。

获得锁的线程用完后一定要释放锁，否则那些苦苦等待锁的线程将永远等待下去，成为死线程。所以我们用try...finally来确保锁一定会被释放。

锁的好处就是确保了某段关键代码只能由一个线程从头到尾完整地执行，坏处当然也很多，首先是阻止了多线程并发执行，包含锁的某段代码实际上只能以单线程模式执行，效率就大大地下降了。其次，由于可以存在多个锁，不同的线程持有不同的锁，并试图获取对方持有的锁时，可能会造成死锁，导致多个线程全部挂起，既不能执行，也无法结束，只能靠操作系统强制终止。